# Technical Analysis Template: sRoC + Stochastic Oscillator

Buy and sell using oversold and overbought levels of the Stochastic oscillator according the trend. 
The trend indicator is the smooth Rate of Change with WMA as a smooth function.

In [ ]:
import qnt.graph as qngraph
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib

import xarray as xr
import pandas as pd
from qnt.stepper import test_strategy
import qnt.forward_looking as qnfl

import xarray.ufuncs as xrf

# Data

In [ ]:
data = qndata.load_data(min_date="2014-01-01", dims=("time", "field", "asset"), forward_order=True)


# Calc output

In [ ]:
SROC_POSITIVE_TREND_LEVEL=0.05
SROC_CLOSE_LEVEL=-0.05

STOCH_OVERBOUGHT_LEVEL=92
STOCH_OVERSOLD_LEVEL=31

wma = qnxrtalib.WMA(data.sel(field='close'), 120)
sroc = qnxrtalib.ROCP(wma, 60)

stoch = qnxrtalib.STOCH(data, 8, 3, 3)
k = stoch.sel(field='slowk')
d = stoch.sel(field='slowd')

data_ext = xr.concat([wma, sroc, k, d], pd.Index(['wma', 'sroc', 'k', 'd'], name='field'))
data_ext = xr.concat([data, data_ext], 'field')

weights = data.isel(time=0, field=0)
weights[:] = 0


def step(data):
    latest = data.isel(time=-1)

    is_liquid = latest.sel(field="is_liquid")
    sroc = latest.sel(field='sroc')
    k = latest.sel(field='k')
    d = latest.sel(field='d')

    need_open = xrf.logical_and(
        sroc > SROC_POSITIVE_TREND_LEVEL, 
        xrf.logical_and(k < STOCH_OVERSOLD_LEVEL, d < STOCH_OVERSOLD_LEVEL)
    )
    need_close = xrf.logical_or(
        sroc < SROC_CLOSE_LEVEL, 
        xrf.logical_and(k > STOCH_OVERBOUGHT_LEVEL, d > STOCH_OVERBOUGHT_LEVEL)
    )

    global weights
    
    weights.loc[need_open] = 1
    weights.loc[need_close] = 0
    
    weights.loc[is_liquid == 0] = 0 # prevention of illiquid assets trading

    return (weights / weights.sum('asset')).fillna(0)


output = test_strategy(data_ext, step=step)

## Stats and plots

In [ ]:
stat = qnstats.calc_stat(data, output, max_periods=252 * 3)
display(stat.to_pandas().tail())

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'equity'].values,  
    color="blue", 
    name="PnL (Equity)", 
    type="log"
)

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'underwater'].values, 
    color="red", 
    name="Underwater Chart", 
    range_max= 0
)

In [ ]:
SR_OFFSET = 252 * 3 + 120 + 60 + 8 * 3 * 3
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas()[SR_OFFSET:], 
    stat.loc[:, 'sharpe_ratio'].values[SR_OFFSET:], 
    color="purple", 
    name="Rolling SR"
)

In [ ]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'bias'].values, 
    color="gray", 
    name="Bias"
)

In [ ]:
qnstats.print_correlation(output, data)

# Checks

In [ ]:
# Use the function from 'qnfl' ensures that no forward-looking
# is taking place. 
def strategy():
    """
    it is the same strtegy, but implemented with xarray
    Entire code of strategy calculation is collected here.
    """
    data = qndata.load_data(min_date="2014-01-01", forward_order=True, dims=("time", "field", "asset"))

    SROC_POSITIVE_TREND_LEVEL=0.05
    SROC_CLOSE_LEVEL=-0.05

    STOCH_OVERBOUGHT_LEVEL=92
    STOCH_OVERSOLD_LEVEL=31

    wma = qnxrtalib.WMA(data.sel(field='close'), 120)
    sroc = qnxrtalib.ROCP(wma, 60)

    stoch = qnxrtalib.STOCH(data, 8, 3, 3)
    k = stoch.sel(field='slowk')
    d = stoch.sel(field='slowd')

    data_ext = xr.concat([wma, sroc, k, d], pd.Index(['wma', 'sroc', 'k', 'd'], name='field'))
    data_ext = xr.concat([data, data_ext], 'field')

    global weights
    weights = data.isel(time=0, field=0)
    weights[:] = 0


    def step(data):
        latest = data.isel(time=-1)

        is_liquid = latest.sel(field="is_liquid")
        sroc = latest.sel(field='sroc')
        k = latest.sel(field='k')
        d = latest.sel(field='d')

        need_open = xrf.logical_and(
            sroc > SROC_POSITIVE_TREND_LEVEL, 
            xrf.logical_and(k < STOCH_OVERSOLD_LEVEL, d < STOCH_OVERSOLD_LEVEL)
        )
        need_close = xrf.logical_or(
            sroc < SROC_CLOSE_LEVEL, 
            xrf.logical_and(k > STOCH_OVERBOUGHT_LEVEL, d > STOCH_OVERBOUGHT_LEVEL)
        )

        global weights

        weights.loc[need_open] = 1
        weights.loc[need_close] = 0

        weights.loc[is_liquid == 0] = 0 # prevention of illiquid assets trading

        return (weights / weights.sum('asset')).fillna(0)


    output = test_strategy(data_ext, step=step)
    
    return output

# This function runs strategy twice on the different periods: 
# the entire data and data the with a cropped last half year.
# After that this function compares outputs. 
# Overlapped outputs must be same.
output_final = qnfl.load_data_calc_output_and_check_forward_looking(strategy)

# Save output

In [ ]:
qndata.write_output(output_final)